In [19]:
import cv2
import numpy as np
import random

# 视频参数
fps = 10
frame_count = fps * 60  # 视频时长60秒
width, height = 640, 480
file = 'moving_digits.mp4'

# 创建视频写入对象
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_writer = cv2.VideoWriter(file, fourcc, fps, (width, height))

def random_color():
    """
    生成随机颜色
    """
    return random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)

def generate_noisy_background(base_color, noise_level=50):
    """
    生成带噪声的背景图像
    :param base_color: 背景颜色 (BGR)
    :param noise_level: 噪声强度
    :return: 带噪声的背景图像
    """
    # 创建基础颜色图像
    background = np.full((height, width, 3), base_color, np.uint8)
    
    # 添加随机噪声
    noise = np.random.randint(0, noise_level, (height, width, 3), dtype=np.uint8)
    noisy_background = cv2.add(background, noise)
    
    return noisy_background

class MovingVideo:
    def __init__(self, digit):
        self.digit = digit
        self.x = random.randint(0, width)
        self.y = random.randint(0, height)
        self.color = random_color()
        self.vx = random.uniform(-3, 3)
        self.vy = random.uniform(-3, 3)
        self.angle = random.randrange(0, 360)
        self.angleV = random.uniform(-5, 5)
        self.size = random.uniform(30, 50)
        self.speed_change = random.uniform(-0.3, 0.3)
        self.out_of_bounds = False  # 标记是否离开边界
        self.frames_out = 0  # 记录离开边界的帧数
        self.max_frames_out = 20  # 离开边界后保持显示的帧数
        self.jitter_angle = random.uniform(-5, 5)  # 新增抖动角度

    def update(self):
        """
        更新数字的位置和状态
        :return: 如果数字仍在帧内，返回 True；否则返回 False
        """
        if self.out_of_bounds:
            self.frames_out += 1
            if self.frames_out > self.max_frames_out:
                return False  # 超过显示时间，完全消失
            return True
        
        # 随机变化速度方向和大小
        direction_change = random.uniform(-0.7, 0.7)
        speed_change = random.uniform(-0.3, 0.3)
        self.vx += speed_change + direction_change
        self.vy += speed_change + direction_change
        
        # 确保速度在一定范围内
        self.vx = max(min(self.vx, 5), -5)
        self.vy = max(min(self.vy, 5), -5)
        
        self.x += self.vx
        self.y += self.vy
        self.angle += self.angleV + self.jitter_angle  # 添加抖动
        
        # 检查边界
        if self.x > width or self.x < 0 or self.y > height or self.y < 0:
            self.out_of_bounds = True
            return True
        
        return True
        
    def draw(self, frame):
        """
        绘制数字到视频帧
        :return: None
        """
        if self.out_of_bounds and self.frames_out > self.max_frames_out:
            return  # 超过显示时间，不再绘制
        
        # 创建一个透明背景的图像
        digit_image = np.zeros((int(self.size), int(self.size), 4), np.uint8)
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = self.size / 40
        color = (int(self.color[0]), int(self.color[1]), int(self.color[2]))
        
        # 绘制数字
        cv2.putText(digit_image, str(self.digit), (int(self.size / 4), int(self.size / 1.5)), font, font_scale, color + (255,), 2, cv2.LINE_AA)

        # 旋转图像
        center = (int(self.size // 2), int(self.size // 2))
        rot_mat = cv2.getRotationMatrix2D(center, self.angle, 1)
        rotated_digit_image = cv2.warpAffine(digit_image, rot_mat, (int(self.size), int(self.size)), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_TRANSPARENT)

        # 获取裁剪区域
        x1, y1 = int(self.x - self.size // 2), int(self.y - self.size // 2)
        x2, y2 = int(self.x + self.size // 2), int(self.y + self.size // 2)
        
        # 确保裁剪区域在帧内
        x1, x2 = max(0, x1), min(x2, width)
        y1, y2 = max(0, y1), min(y2, height)
        
        if x1 < x2 and y1 < y2:
            overlay = frame[y1:y2, x1:x2]
            
            # 计算裁剪区域
            rot_x1 = max(0, -x1)
            rot_y1 = max(0, -y1)
            rot_x2 = min(self.size, width - x1)
            rot_y2 = min(self.size, height - y1)
            
            rotated_digit_image_cropped = rotated_digit_image[int(rot_y1):int(rot_y2), int(rot_x1):int(rot_x2)]
            
            # 确保裁剪区域的尺寸匹配
            if rotated_digit_image_cropped.shape[0] == overlay.shape[0] and rotated_digit_image_cropped.shape[1] == overlay.shape[1]:
                # 分离 Alpha 通道并应用
                alpha_channel = rotated_digit_image_cropped[:, :, 3] / 255.0
                for c in range(3):
                    overlay[:, :, c] = (alpha_channel * rotated_digit_image_cropped[:, :, c] + (1 - alpha_channel) * overlay[:, :, c])

# 创建 MovingVideo 对象
digits = [MovingVideo(random.randint(0, 9)) for _ in range(30)]

# 生成背景颜色
base_color = (50, 50, 50)  # 深灰色背景
noise_level = 25  # 噪声强度
desired_digits_count = 30  # 目标数字数量

def get_direction_towards_screen(x, y, vx, vy):
    """
    获取运动方向朝向屏幕内的速度
    :param x: 当前 x 坐标
    :param y: 当前 y 坐标
    :param vx: 当前 x 方向速度
    :param vy: 当前 y 方向速度
    :return: 新的速度方向 (vx, vy)
    """
    if x < 1:
        vx = random.uniform(1, 3)  # 向右运动
    elif x > width - 1:
        vx = random.uniform(-3, -1)  # 向左运动
    else:
        vx = random.uniform(-3, 3)  # 随机运动

    if y < 1:
        vy = random.uniform(1, 3)  # 向下运动
    elif y > height - 1:
        vy = random.uniform(-3, -1)  # 向上运动
    else:
        vy = random.uniform(-3, 3)  # 随机运动

    return vx, vy

def generate_random_position_near_border(size, width, height):
    """
    在边界内生成一个随机位置，距离边界1像素
    :param size: 数字的大小
    :param width: 帧宽度
    :param height: 帧高度
    :return: 数字的初始 x 和 y 坐标
    """
    border_margin = 1  # 边界内距离设置为1像素
    side = random.choice(['left', 'right', 'top', 'bottom'])
    
    if side == 'left':
        x = border_margin
        y = random.uniform(0, height)
    elif side == 'right':
        x = width - border_margin
        y = random.uniform(0, height)
    elif side == 'top':
        x = random.uniform(0, width)
        y = border_margin
    elif side == 'bottom':
        x = random.uniform(0, width)
        y = height - border_margin
    
    return x, y

for frame_index in range(frame_count):
    frame = generate_noisy_background(base_color, noise_level)
    
    # 更新现有数字
    digits_to_update = []
    for digit in digits:
        if not digit.update():
            digits_to_update.append(digit)
    
    # 替换离开边界的数字
    for digit in digits_to_update:
        digits.remove(digit)
        # 从边界内生成新数字，并将其随机放置在边界内
        new_digit = MovingVideo(random.randint(0, 9))
        new_digit.x, new_digit.y = generate_random_position_near_border(new_digit.size, width, height)
        new_digit.vx, new_digit.vy = get_direction_towards_screen(new_digit.x, new_digit.y, new_digit.vx, new_digit.vy)
        digits.append(new_digit)
    
    # 确保视频中始终有足够的数字
    while len(digits) < desired_digits_count:
        new_digit = MovingVideo(random.randint(0, 9))
        new_digit.x, new_digit.y = generate_random_position_near_border(new_digit.size, width, height)
        new_digit.vx, new_digit.vy = get_direction_towards_screen(new_digit.x, new_digit.y, new_digit.vx, new_digit.vy)
        digits.append(new_digit)
    
    # 绘制所有数字
    for digit in digits:
        digit.draw(frame)
    
    video_writer.write(frame)

video_writer.release()
